In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3

In [ ]:
#establishes file connection
con = sqlite3.connect(r'"C:\Users\andre\OneDrive\Desktop\ds250\data_files\lahmansbaseballdb.sqlite"')

## Elevator pitch
Below you will find results for the data that was requested. This includes information about players that attended BYU-Idaho, as well as information relating to batting averages for many players with more than one, then more than ten at-bats. You will find a list of career batting averages for players with more than 100 at-bats. Lastly, you will find an analysis and a comparison of win rates between the Dodgers and the Giants.

## Question|Task 1

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__

This task requires just an SQL query to retrieve the requested information:

In [ ]:
# selects the above columns; all come from salaries except school id
#school id comes from collegeplaying; salaries joined on college playing by playerID
#where school is byui
#ordered by salary, highest to lowest

id_sql = '''
SELECT DISTINCT

salaries.playerID AS 'Player ID',
(nameFirst || ' ' || nameLast) AS 'Player Name',
salaries.salary AS Salary, 
salaries.yearID AS Year, 
salaries.teamID AS Team,
teams.name AS 'Team Name',
collegeplaying.schoolID AS 'School ID'

FROM 
salaries 
JOIN 
people ON salaries.playerID = people.playerID
JOIN 
collegeplaying ON salaries.playerID = collegeplaying.playerID
JOIN 
teams on teams.teamID = salaries.teamID AND teams.yearID = salaries.yearID


WHERE collegeplaying.schoolID = 'idbyuid'

ORDER BY salary DESC;

'''

df = pd.read_sql_query(id_sql, con)
format_df = df.copy()
format_df['Salary'] = format_df['Salary'].apply(
    lambda x: '${:,.0f}'.format(x)
)

#### Results

In [ ]:
format_df

Displayed above are the required columns, as well as some additional columns with some extra descriptive information.

It appears that two professional players have come from BYU-Idaho: Matt Lindstrom, and Garrett Stephenson. Garrett Stephenson mostly played for the Cardinals, and earned $1.025 million in his highest-earning year. Matt Lindstrom mostly played for the Marlins and the White Sox, and earned up to $4 million.

## Question|Task 2

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats)__

### Task A
- __Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report.__

#### Results

Displayed below are the expected results for the requested query.


In [ ]:
bat_sql = '''
SELECT
b.playerID AS 'Player ID',
(p.nameFirst || ' ' || p.nameLast) AS 'Player Name',
yearID AS 'Year ID',
H as Hits, AB as 'At-bats',
(CAST(H AS FLOAT) / AB) AS 'Batting Average'
FROM batting b
JOIN people p ON p.playerID = b.playerID
WHERE AB >= 1
ORDER BY (CAST(H AS FLOAT) / AB) DESC, b.playerID
LIMIT 5;
'''

bat_df = pd.read_sql_query(bat_sql , con)
bat_df

It appears that the top results of this query contain players that have just 1 at-bat and just 1 hit, giving them a perfect batting average.

### Task B

- __Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results.__

#### Results

Below are the adjusted results.


In [ ]:
bat_sql = '''
SELECT 
b.playerID AS 'Player ID',
(p.nameFirst || ' ' || p.nameLast) AS 'Player Name',
yearID AS 'Year ID',
H as Hits, AB as 'At-bats',
(CAST(H AS FLOAT) / AB) AS 'Batting Average'
FROM batting b
JOIN people p ON p.playerID = b.playerID
WHERE AB >= 10
ORDER BY (CAST(H AS FLOAT) / AB) DESC, b.playerID
LIMIT 5;
'''

bat_df = pd.read_sql_query(bat_sql , con)
bat_df

Although these results will be more reflective of the players' actual abilities, the Law of Large Numbers tells us we should be looking for higher numbers of at-bats than 10 to accurately represent the players' abilities.

### Task C
- __Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results.__

#### Results

Displayed are the results of the query.

In [ ]:
career_sql = '''
SELECT 
b.playerID AS 'Player ID',
(p.nameFirst || ' ' || p.nameLast) AS 'Player Name',
MIN(b.yearID) AS 'Rookie Year',
SUM(b.H) AS 'Career Hits',
SUM(b.AB) AS 'Career At-Bats',
CAST(SUM(b.H) AS FLOAT) / SUM(AB) AS 'Career Batting Average'

FROM 
batting b
JOIN
people p ON p.playerID = b.playerID

WHERE 
AB >= 100

GROUP BY 
b.playerID

ORDER BY 
'Career Batting Average' DESC
LIMIT 5;
'''

career_df = pd.read_sql_query(career_sql, con)
career_df

## Question|Task 3
__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph using Plotly Express to visualize the comparison. What do you learn?__

I decided to compare the age-old rivalry between the Dodgers and the Giants. The point of a baseball game is to win; beyond this goal, not much else really matters more to the average fan. Being a Dodgers fan, I hope that the Dodgers come out on top with a higher win percentage. I decided to only use each team's current name and city, because as a fan of the Los Angeles Dodgers, that is what matters most to me.


#### Results

In [ ]:
wins_sql = '''
SELECT

name AS Name,
sum(W) AS 'Total Wins',
sum(G) AS 'Total Games',
(CAST(sum(W) AS FLOAT) / sum(G)) AS 'Win Percentage'

FROM teams

WHERE name IN ('Los Angeles Dodgers' , 'San Francisco Giants')

GROUP BY name

ORDER BY 'Win Percentage';
'''

wins_df = pd.read_sql_query(wins_sql, con)

#applies visual formatting
for i in ['Total Wins', 'Total Games']:
  wins_df[i] = wins_df[i].astype(str).apply(
      lambda x: '{:,}'.format(int(x)))

fig = px.histogram(
  wins_df,
  x = 'Name',
  y = 'Win Percentage',
  color = {'Los Angeles Dodgers': 'blue' , 'San Francisco Giants': 'red'},
  labels = {'Name': 'Team' , 'Win Percentage': 'Win Percentage'},
  title = 'Win Percentage By Team'
).update_layout(
  title_x = 0.5
)

#applies further formatting
wins_df['Win Percentage'] = (wins_df['Win Percentage'] * 100).round(2).astype(str) + '%'

fig.show()

In [ ]:
wins_df

The current Dodgers have a better win percentage than the current Giants! My day is made.